In [1]:
#connect to drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Importing important Libraries

In [6]:
#importing basic libraries

import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt

In [2]:
#importing models

from xgboost import XGBRegressor

In [3]:
#importing evaluation metrics

from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error,mean_squared_log_error

In [4]:
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [7]:
#loading the data
## true unprocessed csv file

df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/true_car_listings.csv")

df.shape

(852122, 8)

In [8]:
df.head()

,Price,Year,Mileage,City,State,Vin,Make,Model
0,8995,2014,35725,El Paso,TX,19VDE2E53EE000083,Acura,ILX6-Speed
1,10888,2013,19606,Long Island City,NY,19VDE1F52DE012636,Acura,ILX5-Speed
2,8995,2013,48851,El Paso,TX,19VDE2E52DE000025,Acura,ILX6-Speed
3,10999,2014,39922,Windsor,CO,19VDE1F71EE003817,Acura,ILX5-Speed
4,14799,2016,22142,Lindon,UT,19UDE2F32GA001284,Acura,ILXAutomatic


# Data Cleaning:

In [9]:
#Vin(Unique car ID) is unecessary column so we are dropping it
df.drop('Vin',axis=1,inplace=True)
df.shape

(852122, 7)

Removing duplicates

In [10]:
#checking duplicate data that might be present in the dataset

dfdup=df[df.duplicated()]
dfdup

,Price,Year,Mileage,City,State,Make,Model
314,22000,2017,10,Chicago,IL,Acura,ILXAutomatic
1259,23566,2017,16,Larchmont,NY,Acura,ILXPremium
6258,36000,2018,5,Littleton,CO,Acura,RDXAWD
6356,33900,2017,4250,Salt Lake City,UT,Acura,RDXAWD
7180,38275,2018,5,Littleton,CO,Acura,RDXAWD
...,...,...,...,...,...,...,...
819060,36998,2014,35370,Houston,TX,Toyota,4Runner4x4
819874,32018,2017,6,Mechanicsville,VA,Toyota,Tacoma2WD
824387,16995,2017,15,San Antonio,TX,Volkswagen,Passat1.8T
825336,16996,2017,7787,San Antonio,TX,Volkswagen,Passat1.8T


In [11]:
#checking shape of dataset
df.shape

(852122, 7)

In [12]:
#removing duplicated rows from the dataset
df.drop_duplicates(inplace=True)

#again checking for shape of the now data
df.shape

(851568, 7)

Removing null values

In [13]:
#checking for missing values

df.isna().sum()

Price      0
Year       0
Mileage    0
City       0
State      0
Make       0
Model      0
dtype: int64

In [14]:
#checking all datatypes

df.dtypes

Price       int64
Year        int64
Mileage     int64
City       object
State      object
Make       object
Model      object
dtype: object

In [15]:
#checking the variables

df.columns

Index(['Price', 'Year', 'Mileage', 'City', 'State', 'Make', 'Model'], dtype='object')

In [16]:
#getting additional info about the dataset

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 851568 entries, 0 to 852121
Data columns (total 7 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   Price    851568 non-null  int64 
 1   Year     851568 non-null  int64 
 2   Mileage  851568 non-null  int64 
 3   City     851568 non-null  object
 4   State    851568 non-null  object
 5   Make     851568 non-null  object
 6   Model    851568 non-null  object
dtypes: int64(3), object(4)
memory usage: 52.0+ MB


In [17]:
#further describing our dataset

df.describe()

,Price,Year,Mileage
count,851568.000000,851568.000000,8.515680e+05
mean,21458.705582,2013.286910,5.253862e+04
std,13594.437056,3.414859,4.198370e+04
min,1500.000000,1997.000000,5.000000e+00
25%,13000.000000,2012.000000,2.386400e+04
50%,18500.000000,2014.000000,4.027900e+04
75%,26995.000000,2016.000000,7.221800e+04
max,499500.000000,2018.000000,2.856196e+06


In [18]:
#checking number of unique values in the columns

df.nunique()

Price       47124
Year           22
Mileage    158836
City         2553
State          59
Make           58
Model        2736
dtype: int64

In [19]:
df.shape

(851568, 7)

In [20]:

df.columns

Index(['Price', 'Year', 'Mileage', 'City', 'State', 'Make', 'Model'], dtype='object')

In [21]:
df['Mileage'].dtypes

dtype('int64')

In [22]:
#Dropping City and State as they have minimal correlation for prediction of price
df.drop(['City','State'],axis=1,inplace=True)

In [23]:
df.head()

,Price,Year,Mileage,Make,Model
0,8995,2014,35725,Acura,ILX6-Speed
1,10888,2013,19606,Acura,ILX5-Speed
2,8995,2013,48851,Acura,ILX6-Speed
3,10999,2014,39922,Acura,ILX5-Speed
4,14799,2016,22142,Acura,ILXAutomatic


In [24]:
df.shape

(851568, 5)

In [25]:
# Concatenate 'Make' and 'Model' columns into a new column 'Model'
df['Model'] = df['Make'] + ' ' + df['Model']

# Display the resulting DataFrame
print(df)

        Price  Year  Mileage   Make               Model
0        8995  2014    35725  Acura    Acura ILX6-Speed
1       10888  2013    19606  Acura    Acura ILX5-Speed
2        8995  2013    48851  Acura    Acura ILX6-Speed
3       10999  2014    39922  Acura    Acura ILX5-Speed
4       14799  2016    22142  Acura  Acura ILXAutomatic
...       ...   ...      ...    ...                 ...
852117  63215  2017        9  Volvo         Volvo S90T6
852118  72260  2017     3201  Volvo        Volvo XC90T6
852119  55999  2016    28941  Volvo       Volvo XC90AWD
852120  60240  2017     3005  Volvo           Volvo V90
852121  76995  2017     2502  Volvo        Volvo XC90T8

[851568 rows x 5 columns]


In [14]:
#Saving cleaned data
##DO NOT RUN AGAIN
#df.to_csv("/content/drive/MyDrive/Colab Notebooks/cars_clean_data.csv",index= False, header=True)

In [26]:
df.head()

,Price,Year,Mileage,Make,Model
0,8995,2014,35725,Acura,Acura ILX6-Speed
1,10888,2013,19606,Acura,Acura ILX5-Speed
2,8995,2013,48851,Acura,Acura ILX6-Speed
3,10999,2014,39922,Acura,Acura ILX5-Speed
4,14799,2016,22142,Acura,Acura ILXAutomatic


In [27]:
#seperating the independant and dependant variables
x = df.drop('Price',axis=1)
y = df['Price']

x.shape,y.shape

((851568, 4), (851568,))

In [28]:
#using train test split
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=777,test_size=.30)

In [29]:
x.head()

,Year,Mileage,Make,Model
0,2014,35725,Acura,Acura ILX6-Speed
1,2013,19606,Acura,Acura ILX5-Speed
2,2013,48851,Acura,Acura ILX6-Speed
3,2014,39922,Acura,Acura ILX5-Speed
4,2016,22142,Acura,Acura ILXAutomatic


In [30]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Define transformers
encoder = OneHotEncoder(handle_unknown='ignore')
encoder.fit(x[['Make','Model']])
scaler = StandardScaler()
scaler.fit(x.loc[:,['Year','Mileage']])
scaler.transform(x.loc[:,['Year','Mileage']])

# Specify columns for each transformer
encoder_columns = ['Make', 'Model']
scaler_columns = ['Year', 'Mileage']

# Create the column transformer
column_trans = make_column_transformer(
    (encoder, encoder_columns),
    (scaler, scaler_columns)
)

In [31]:
#XGBoost

xgb=XGBRegressor()
#pipe=make_pipeline(trf1,trf2,xgb)
pipe=make_pipeline(column_trans,xgb)
pipe.fit(x_train,y_train)
y_pred=pipe.predict(x_test)
print(r2_score(y_test,y_pred)*100)

81.7726270552755


In [32]:
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))